In [ ]:
import pandas as pd
df_history = pd.read_csv('D:\\OneDrive\\Desktop\\Babson\\Customer_In Stock_Dataset_Final.csv',encoding= 'unicode_escape')
df_forward = pd.read_csv('D:\\OneDrive\\Desktop\\Babson\\Forward Coverage Dataset Final.csv',encoding= 'unicode_escape' )
df_forward_1 = df_forward[['Date', 'Forecast', 'qty_fulfilled_oh', 'qty_fulfilled_oo', 'qty_fulfilled_if', 'qty_fulfilled_l2']]
df_forward['Actual sales'] = df_forward['Forecast']
df_forward['Covered sales'] = df_forward['qty_fulfilled_oh'] + df_forward['qty_fulfilled_oo'] + df_forward['qty_fulfilled_if'] + df_forward['qty_fulfilled_l2']
df_forward['Missed value'] = df_forward['Actual sales'] - df_forward['Covered sales']
df_forward['instock%'] = df_forward['Covered sales'] / df_forward['Actual sales']
df_forward['Demand'] = df_forward['Forecast'] / df_forward['WholesaleCost']
df_forward_sub = df_forward.loc[:, ['Date', 'SKU', 'suid', 'Country', 'ItemClass', 'ItemMarketingCategory', 'ItemShipClass', 'Actual sales', 'Covered sales', 'Missed value', 'instock%', 'Demand']]
df_foward_sub = df_forward_sub[df_forward_sub['Country'] == 'United States']
df_history = df_history[df_history['Country'] == 'United States']
history = df_history.groupby(['ItemMarketingCategory'])['suid'].nunique()
forward = df_foward_sub.groupby(['ItemMarketingCategory'])['suid'].nunique()
df_history = pd.DataFrame(history.values, index = history.index, columns = ['SupplierCount_his'])
df_forward = pd.DataFrame(forward.values, index = forward.index, columns = ['SupplierCount_for'])
result = pd.concat([df_forward, df_history], axis=1, join="inner")
result['%change'] = (result['SupplierCount_for'] - result['SupplierCount_his']) / result['SupplierCount_his'] * 100
result.sort_values('%change', ascending = False, inplace = True)
result
priority_Score = pd.read_csv('D:\\OneDrive\\Desktop\\babson\\final_positive_data.csv')
priority_Score = priority_Score.loc[:,['SKU', 'Priority_score', 'ItemMarketingCategory']]

priority_Score = priority_Score.groupby('ItemMarketingCategory')['Priority_score'].mean()
priority_Score = pd.DataFrame(priority_Score.values, index = priority_Score.index, columns = ['priority_score'])
result = pd.concat([result, priority_Score], axis = 1 ,join = 'inner')
result['Metric_After_Priority_Score'] = result['%change'] * result['priority_score']
result.sort_values('Metric_After_Priority_Score', inplace = True)
result.reset_index(inplace = True)

import seaborn as sns
import matplotlib.pyplot as plt
result.head()
result.sort_values('%change', inplace = True)
ax = sns.barplot(x = 'ItemMarketingCategory', y = '%change', data = result, palette='Reds_r')

plt.xticks(rotation=90, fontsize = 7)
plt.ylabel('Change%')
plt.title('Change% of #Suppliers for Categories', fontsize = 17)
# norm = plt.Normalize(result['%change'].min(), result['%change'].max())
# sm = plt.cm.ScalarMappable(cmap="Reds", norm=norm)
# cbar = plt.colorbar(sm)
plt.xlabel('x axis:different item class categories')
png1 = ax.get_figure()
plt.savefig('D:\\OneDrive\\Desktop\\Babson\\plot.jpeg', dpi=300, bbox_inches='tight')